In [47]:
import pandas as pd 
import numpy as np 

from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import BoxScoreAdvancedV3
from nba_api.stats.endpoints.teamestimatedmetrics import TeamEstimatedMetrics
from nba_api.stats.library.parameters import LeagueID

In [48]:
# Get team ID for the Toronto Raptors
team_dict = teams.get_teams()
raptors = [team for team in team_dict if team['abbreviation'] == 'TOR'][0]
raptors_id = raptors['id']

# Get team ID for matchup
opp_abbreviation = 'BOS'
opponent = [team for team in team_dict if team['abbreviation'] == opp_abbreviation][0]
opp_id = opponent['id']

# Set season
chosen_season='2023-24'

# Set game type
game_type = 'Regular Season'

In [52]:
# Get all of the Raptors games
raptors_game_log = teamgamelog.TeamGameLog(
    team_id=raptors_id,
    season=chosen_season,
    season_type_all_star = game_type,
    league_id_nullable = ''
)

# Access the game log data
raptors_games_data = raptors_game_log.team_game_log.get_dict()['data']

# Create a DataFrame with labeled columns
columns = raptors_game_log.expected_data['TeamGameLog']
raptors_df = pd.DataFrame(raptors_games_data, columns=columns)


# Print the head of the DataFrame
print(raptors_df.head())

      Team_ID     Game_ID     GAME_DATE      MATCHUP WL   W   L  W_PCT  MIN  \
0  1610612761  0022300923  MAR 09, 2024    TOR @ POR  L  23  41  0.359  265   
1  1610612761  0022300905  MAR 07, 2024    TOR @ PHX  L  23  40  0.365  240   
2  1610612761  0022300890  MAR 05, 2024  TOR vs. NOP  L  23  39  0.371  240   
3  1610612761  0022300875  MAR 03, 2024  TOR vs. CHA  W  23  38  0.377  240   
4  1610612761  0022300860  MAR 01, 2024  TOR vs. GSW  L  22  38  0.367  240   

   FGM  ...  FT_PCT  OREB  DREB  REB  AST  STL  BLK  TOV  PF  PTS  
0   43  ...   0.842     9    34   43   29    9    6   20  18  118  
1   43  ...   0.733     9    27   36   30    5    2    9  20  113  
2   36  ...   0.591    11    34   45   21    6    7   11  11   98  
3   42  ...   0.778    11    34   45   27    8    6   14  16  111  
4   41  ...   0.647    11    36   47   31    9    2   14  14  105  

[5 rows x 27 columns]


In [4]:
# Drop the unneccesary information
columns_to_drop = ['Team_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'W', 'L', 'WL', 'W_PCT', 'MIN']

# Drop the specified columns
filtered_raptors_games_data = raptors_df.drop(columns=columns_to_drop)

# Calculate the average stats
raptors_average_stats = filtered_raptors_games_data.mean()

# Convert the result to a DataFrame
raptors_average_stats_df = pd.DataFrame(raptors_average_stats).T

# Print the DataFrame with average stats
print(raptors_average_stats_df)

# Add to data
data = pd.concat([data, raptors_average_stats_df], axis=1)


         FGM        FGA    FG_PCT      FG3M       FG3A   FG3_PCT        FTM  \
0  42.952381  90.428571  0.475651  11.84127  33.349206  0.354667  16.365079   

         FTA    FT_PCT       OREB       DREB        REB        AST       STL  \
0  21.952381  0.746111  11.269841  32.396825  43.666667  29.253968  7.507937   

        BLK        TOV         PF         PTS  
0  4.888889  12.698413  18.285714  114.111111  


In [5]:
# Filter rows where 'MATCHUP' column ends with opponent abbreviation
opp_games = raptors_df[
    raptors_df['MATCHUP'].str.endswith(opp_abbreviation)
]

# Drop the unneccesary information
columns_to_drop = ['Team_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'W', 'L', 'WL', 'W_PCT', 'MIN']

# Drop the specified columns
opp_games = opp_games.drop(columns=columns_to_drop)

# Calculate the average stats against the opponent
average_stats_opp = opp_games.mean()

# Convert the result to a DataFrame
average_stats_opp_df = pd.DataFrame(average_stats_opp).T

#Rename columns for matchup
average_stats_opp_df = average_stats_opp_df.add_prefix(f'vs{opp_abbreviation}_')

# Print the DataFrame with average stats against opponent
print(average_stats_opp_df)

# Add to data
data = pd.concat([data, average_stats_opp_df], axis=1)

   vsBOS_FGM  vsBOS_FGA  vsBOS_FG_PCT  vsBOS_FG3M  vsBOS_FG3A  vsBOS_FG3_PCT  \
0      39.75       92.0       0.43275       10.75        36.5        0.28625   

   vsBOS_FTM  vsBOS_FTA  vsBOS_FT_PCT  vsBOS_OREB  vsBOS_DREB  vsBOS_REB  \
0       13.0      17.75        0.7325         7.5        34.5       42.0   

   vsBOS_AST  vsBOS_STL  vsBOS_BLK  vsBOS_TOV  vsBOS_PF  vsBOS_PTS  
0       27.0        7.0        2.5        8.0     15.25     103.25  


In [6]:
# Select the last 5 games
last_5_games = raptors_df.head(5)

# Drop the unneccesary information
columns_to_drop = ['Team_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'W', 'L', 'WL', 'W_PCT', 'MIN']

# Drop the specified columns
last_5_games = last_5_games.drop(columns=columns_to_drop)

# Print the DataFrame of the last 5 games
print(last_5_games)

   FGM  FGA  FG_PCT  FG3M  FG3A  FG3_PCT  FTM  FTA  FT_PCT  OREB  DREB  REB  \
0   43   94   0.457    16    44    0.364   11   15   0.733     9    27   36   
1   36   93   0.387    13    44    0.295   13   22   0.591    11    34   45   
2   42   91   0.462    13    38    0.342   14   18   0.778    11    34   45   
3   41   96   0.427    12    45    0.267   11   17   0.647    11    36   47   
4   46   98   0.469    13    35    0.371   20   30   0.667    12    32   44   

   AST  STL  BLK  TOV  PF  PTS  
0   30    5    2    9  20  113  
1   21    6    7   11  11   98  
2   27    8    6   14  16  111  
3   31    9    2   14  14  105  
4   34   11    3   11  21  125  


In [7]:
# Calculate the average stats for the last 5 games
average_stats_last_5 = last_5_games.mean()

# Convert the result to a DataFrame
average_stats_last_5_df = pd.DataFrame(average_stats_last_5).T  # T is used to transpose the result to a single-row DataFrame

# Add Last_5 to each column name
average_stats_last_5_df = average_stats_last_5_df.add_prefix('last_5_')

# Print the DataFrame with average stats for the last 5 games
print(average_stats_last_5_df)

# Add to data
data = pd.concat([data, average_stats_last_5_df], axis=1)

   last_5_FGM  last_5_FGA  last_5_FG_PCT  last_5_FG3M  last_5_FG3A  \
0        41.6        94.4         0.4404         13.4         41.2   

   last_5_FG3_PCT  last_5_FTM  last_5_FTA  last_5_FT_PCT  last_5_OREB  \
0          0.3278        13.8        20.4         0.6832         10.8   

   last_5_DREB  last_5_REB  last_5_AST  last_5_STL  last_5_BLK  last_5_TOV  \
0         32.6        43.4        28.6         7.8         4.0        11.8   

   last_5_PF  last_5_PTS  
0       16.4       110.4  


In [8]:
# Set the parameters for the API request
league_id = LeagueID.default
season = chosen_season  # Replace with the desired season
season_type = game_type

# Create an instance of TeamEstimatedMetrics with the specified parameters
raptors_estimated_metrics = TeamEstimatedMetrics(
    league_id=league_id,
    season=season,
    season_type=season_type,
)

# Get the estimated metrics data
estimated_metrics_data = raptors_estimated_metrics.team_estimated_metrics.get_dict()['data']

# Add column names
column_names = TeamEstimatedMetrics.expected_data['TeamEstimatedMetrics']

# Convert to dataframe with appropriate column names
estimated_metrics_df = pd.DataFrame(estimated_metrics_data, columns=column_names)

In [9]:
# Create a filtered dataframe for the Raptors data
raptors_estimated_metrics_df = estimated_metrics_df[
    (estimated_metrics_df['TEAM_NAME'] == 'Toronto Raptors') | (estimated_metrics_df['TEAM_ID'] == raptors_id)
]

# List of columns to drop
columns_to_drop = ['TEAM_NAME', 'TEAM_ID', 'GP', 'W', 'L', 'W_PCT', 'MIN']

# Drop the specified columns
raptors_estimated_metrics_df = raptors_estimated_metrics_df.drop(columns=columns_to_drop)

# Add prefix to column name
raptors_estimated_metrics_df = raptors_estimated_metrics_df.add_prefix('TOR_')

# Print the DataFrame with Raptors' stats
print(raptors_estimated_metrics_df)

# Add to data
data = data.reset_index(drop=True)
raptors_estimated_metrics_df = raptors_estimated_metrics_df.reset_index(drop=True)


data = pd.concat([data, raptors_estimated_metrics_df], axis=1)

    TOR_E_OFF_RATING  TOR_E_DEF_RATING  TOR_E_NET_RATING  TOR_E_PACE  \
12             111.7             115.0              -3.2       101.7   

    TOR_E_AST_RATIO  TOR_E_OREB_PCT  TOR_E_DREB_PCT  TOR_E_REB_PCT  \
12             20.3            0.25           0.748          0.494   

    TOR_E_TM_TOV_PCT  TOR_GP_RANK  ...  TOR_MIN_RANK  TOR_E_OFF_RATING_RANK  \
12              0.13           11  ...            14                     21   

    TOR_E_DEF_RATING_RANK  TOR_E_NET_RATING_RANK  TOR_E_AST_RATIO_RANK  \
12                     23                     23                     4   

    TOR_E_OREB_PCT_RANK  TOR_E_DREB_PCT_RANK  TOR_E_REB_PCT_RANK  \
12                   14                   21                  20   

    TOR_E_TM_TOV_PCT_RANK  TOR_E_PACE_RANK  
12                     15               13  

[1 rows x 23 columns]


In [10]:
# Create a filtered dataframe for the opponent's data
opp_estimated_metrics_df = estimated_metrics_df[
    (estimated_metrics_df['TEAM_ID'] == opp_id)
]

# List of columns to drop
columns_to_drop = ['TEAM_NAME', 'TEAM_ID', 'GP', 'W', 'L', 'W_PCT', 'MIN']

# Drop the specified columns
opp_estimated_metrics_df = opp_estimated_metrics_df.drop(columns=columns_to_drop)

opp_estimated_metrics_df = opp_estimated_metrics_df.add_prefix(f'{opp_abbreviation}_')

# Print the DataFrame with opponent's stats
print(opp_estimated_metrics_df)

# Add to data
data = data.reset_index(drop=True)
opp_estimated_metrics_df = opp_estimated_metrics_df.reset_index(drop=True)

data = pd.concat([data, opp_estimated_metrics_df], axis=1)

    BOS_E_OFF_RATING  BOS_E_DEF_RATING  BOS_E_NET_RATING  BOS_E_PACE  \
19             119.4             108.7              10.7       100.3   

    BOS_E_AST_RATIO  BOS_E_OREB_PCT  BOS_E_DREB_PCT  BOS_E_REB_PCT  \
19             18.9           0.246           0.767           0.52   

    BOS_E_TM_TOV_PCT  BOS_GP_RANK  ...  BOS_MIN_RANK  BOS_E_OFF_RATING_RANK  \
19             0.122           26  ...            29                      1   

    BOS_E_DEF_RATING_RANK  BOS_E_NET_RATING_RANK  BOS_E_AST_RATIO_RANK  \
19                      3                      1                    17   

    BOS_E_OREB_PCT_RANK  BOS_E_DREB_PCT_RANK  BOS_E_REB_PCT_RANK  \
19                   16                    8                   4   

    BOS_E_TM_TOV_PCT_RANK  BOS_E_PACE_RANK  
19                      4               20  

[1 rows x 23 columns]


In [11]:
print(data)

         FGM        FGA    FG_PCT      FG3M       FG3A   FG3_PCT        FTM  \
0  42.952381  90.428571  0.475651  11.84127  33.349206  0.354667  16.365079   

         FTA    FT_PCT       OREB  ...  BOS_MIN_RANK  BOS_E_OFF_RATING_RANK  \
0  21.952381  0.746111  11.269841  ...            29                      1   

   BOS_E_DEF_RATING_RANK  BOS_E_NET_RATING_RANK  BOS_E_AST_RATIO_RANK  \
0                      3                      1                    17   

   BOS_E_OREB_PCT_RANK  BOS_E_DREB_PCT_RANK  BOS_E_REB_PCT_RANK  \
0                   16                    8                   4   

   BOS_E_TM_TOV_PCT_RANK  BOS_E_PACE_RANK  
0                      4               20  

[1 rows x 100 columns]
